In [1]:
import FABRIC

params = FABRIC.loadArgs('./params/params.yaml')
# FABRIC.dirSweep('/esplabdata/FABRIC/raw')

data, _, _ = FABRIC.loadbsf('/esplabdata/FABRIC/raw/MP/1008_MP_POST_BSC1.bsf', plot = False)

In [2]:
data

array([[ 1.44250490e+00, -1.52435305e-01,  7.56687511e+02,
        -1.34252935e+01,  5.79907251e+00, -4.72534200e+00],
       [ 1.40161135e+00,  1.62673953e-01,  7.55218761e+02,
        -1.38437506e+01,  5.95727564e+00, -4.68774434e+00],
       [ 1.34185793e+00, -5.08728035e-01,  7.56500011e+02,
        -1.37075201e+01,  5.77319360e+00, -4.68798848e+00],
       ...,
       [ 4.54345710e+00, -6.11938486e+00,  7.54562511e+02,
         4.50732441e+00, -3.11904920e-01, -4.30908221e+00],
       [ 4.41357429e+00, -6.06835947e+00,  7.51500011e+02,
         4.81835958e+00, -5.46295189e-01, -4.26489276e+00],
       [ 4.72265632e+00, -6.07641611e+00,  7.54750011e+02,
         4.63549824e+00, -2.82485974e-01, -4.29980487e+00]])